In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from gptchem.data import get_qmug_data
from gptchem.formatter import InverseDesignFormatter
from gptchem.tuner import Tuner
from gptchem.extractor import InverseExtractor
from gptchem.querier import Querier

import numpy as np 

In [3]:
num_samples = 100

In [ ]:
def generate_desired_dist(mean: float = 5.0, width: float = 0.2, num_points: int = 100):
    return np.random.normal(mean, width, size=num_points)

In [7]:
formatter = InverseDesignFormatter(
    representation_column="SMILES",
    property_columns=["GFN2_HOMO_LUMO_GAP_mean_ev"],
    property_names=["bandgap"],
    num_digits=1,
)

In [9]:
tuner = Tuner()

# Iteration 0

In [2]:
data = get_qmug_data()

In [4]:
train_set = data.sample(num_samples)

In [8]:
formatted_0 = formatter(train_set)

In [10]:
tune_res_0 = tuner(formatted_0)

Upload progress: 100%|██████████| 45.0k/45.0k [00:00<00:00, 21.1Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/05_inverse/bandgap/biasing/manual/out/20230209_170232/train.jsonl: file-nYMwTmPBskIID3jKNgtKtdxf
